In [2]:
import pandas as pd
from itertools import product

In [3]:
df_original = pd.read_csv("sales_train.csv")
df_original.columns

Index(['date', 'date_block_num', 'shop_id', 'item_id', 'item_price',
       'item_cnt_day'],
      dtype='object')

In [4]:
df_draft = df_original[["date_block_num", "shop_id", "item_id", "item_cnt_day"]].copy()
df_draft = df_draft.groupby(["date_block_num", "item_id", "shop_id"], as_index=False).sum()
df_result = pd.DataFrame()
for i in range(34):
    df_date_block = df_draft[df_draft["date_block_num"] == i][["date_block_num", "item_id", "shop_id"]]
    df_crossed = pd.DataFrame(list(product([i], df_date_block["item_id"].unique(), df_date_block["shop_id"].unique())),
                               columns=["date_block_num", "item_id", "shop_id"])
    df_result = pd.concat([df_result, df_crossed], ignore_index=True)
df_result = pd.merge(df_result, df_draft, on=["date_block_num", "item_id", "shop_id"], how="left")
df_result["item_cnt_day"].fillna(0, inplace=True)
df_result

,date_block_num,item_id,shop_id,item_cnt_day
0,0,19,25,1.0
1,0,19,1,0.0
2,0,19,2,0.0
3,0,19,10,0.0
4,0,19,19,0.0
...,...,...,...,...
10913845,33,22167,34,0.0
10913846,33,22167,41,0.0
10913847,33,22167,48,0.0
10913848,33,22167,9,0.0


In [8]:
df_sum = df_result.groupby(["date_block_num", "item_id", "shop_id"], as_index=False)["item_cnt_day"].sum()

,date_block_num,item_id,shop_id,item_cnt_day
10794398,33,11373,12,2253.0
10545641,32,9249,55,1644.0
10545567,32,9248,12,1644.0
4471622,11,20949,31,1305.0
9341405,27,3730,55,1242.0
...,...,...,...,...
724919,1,21323,28,-4.0
6986169,19,11367,12,-5.0
18549,0,1593,12,-6.0
18414,0,1590,12,-13.0


In [19]:
df_lag = df_sum.copy()
df_lag["lag_1"] = df_lag["date_block_num"] + 1

df_lag = pd.merge(df_lag, df_sum, left_on=["lag_1", "item_id", "shop_id"], right_on=["date_block_num", "item_id", "shop_id"], how="inner")
df_lag = df_lag.sort_values("item_cnt_day_x", ascending=False)
df_lag

,date_block_num_x,item_id,shop_id,item_cnt_day_x,lag_1,date_block_num_y,item_cnt_day_y
3659843,11,20949,31,1305.0,12,12,899.0
7416771,27,3730,55,1242.0,28,28,2.0
6731599,23,20949,31,1209.0,24,24,743.0
6803663,24,5917,55,1127.0,25,25,5.0
6201671,21,19437,55,1074.0,22,22,3.0
...,...,...,...,...,...,...,...
3116123,10,3732,12,-4.0,11,11,6.0
608389,1,21323,28,-4.0,2,2,3.0
15624,0,1593,12,-6.0,1,1,-1.0
15579,0,1590,12,-13.0,1,1,1.0
